In [1]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cross_validation import KFold
from sklearn.model_selection import learning_curve
import matplotlib.style as style
style.use('fivethirtyeight')
%matplotlib inline

f=open('AviationData.txt')

aviation_data=f.readlines()
aviation_list=[]
lax=[]

for line in aviation_data:
    aviation_list.append(line.split(" | "))

for line in aviation_list:
    if 'LAX94LA336' in line:
        lax.append(line)
print(lax)

/Users/brendenrossin/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[['20001218X45447', 'Accident', 'LAX94LA336', '07/19/1962', 'BRIDGEPORT, CA', 'United States', '', '', '', '', 'Fatal(4)', 'Destroyed', '', 'N5069P', 'PIPER', 'PA24-180', 'No', '1', 'Reciprocating', '', '', 'Personal', '', '4', '0', '0', '0', 'UNK', 'UNKNOWN', 'Probable Cause', '09/19/1996', '\n']]


In [2]:
print(aviation_list[0:5])

[['Event Id', 'Investigation Type', 'Accident Number', 'Event Date', 'Location', 'Country', 'Latitude', 'Longitude', 'Airport Code', 'Airport Name', 'Injury Severity', 'Aircraft Damage', 'Aircraft Category', 'Registration Number', 'Make', 'Model', 'Amateur Built', 'Number of Engines', 'Engine Type', 'FAR Description', 'Schedule', 'Purpose of Flight', 'Air Carrier', 'Total Fatal Injuries', 'Total Serious Injuries', 'Total Minor Injuries', 'Total Uninjured', 'Weather Condition', 'Broad Phase of Flight', 'Report Status', 'Publication Date', '\n'], ['20180719X01751', 'Accident', 'CEN18LA281', '07/19/2018', 'Shreveport, LA', 'United States', '32.540277', '-93.745000', 'DTN', 'Shreveport Executive', 'Non-Fatal', 'Substantial', 'Airplane', 'N7080F', 'PIPER', 'PA28', 'No', '1', 'Reciprocating', 'Part 91: General Aviation', '', 'Personal', '', '', '', '', '1', 'VMC', 'MANEUVERING', 'Preliminary', '07/23/2018', '\n'], ['20180717X23051', 'Accident', 'GAA18CA428', '07/17/2018', 'Nashville, TN', 'U

In [3]:
aviation = [i[:-1] for i in aviation_list]

In [4]:
aviation[0:5]

[['Event Id',
  'Investigation Type',
  'Accident Number',
  'Event Date',
  'Location',
  'Country',
  'Latitude',
  'Longitude',
  'Airport Code',
  'Airport Name',
  'Injury Severity',
  'Aircraft Damage',
  'Aircraft Category',
  'Registration Number',
  'Make',
  'Model',
  'Amateur Built',
  'Number of Engines',
  'Engine Type',
  'FAR Description',
  'Schedule',
  'Purpose of Flight',
  'Air Carrier',
  'Total Fatal Injuries',
  'Total Serious Injuries',
  'Total Minor Injuries',
  'Total Uninjured',
  'Weather Condition',
  'Broad Phase of Flight',
  'Report Status',
  'Publication Date'],
 ['20180719X01751',
  'Accident',
  'CEN18LA281',
  '07/19/2018',
  'Shreveport, LA',
  'United States',
  '32.540277',
  '-93.745000',
  'DTN',
  'Shreveport Executive',
  'Non-Fatal',
  'Substantial',
  'Airplane',
  'N7080F',
  'PIPER',
  'PA28',
  'No',
  '1',
  'Reciprocating',
  'Part 91: General Aviation',
  '',
  'Personal',
  '',
  '',
  '',
  '',
  '1',
  'VMC',
  'MANEUVERING',
  '

In [5]:
headers = aviation.pop(0)
df = pd.DataFrame(aviation, columns=headers)

In [6]:
df.head()

,Event Id,Investigation Type,Accident Number,Event Date,Location,Country,Latitude,Longitude,Airport Code,Airport Name,...,Purpose of Flight,Air Carrier,Total Fatal Injuries,Total Serious Injuries,Total Minor Injuries,Total Uninjured,Weather Condition,Broad Phase of Flight,Report Status,Publication Date
0,20180719X01751,Accident,CEN18LA281,07/19/2018,"Shreveport, LA",United States,32.540277,-93.745000,DTN,Shreveport Executive,...,Personal,,,,,1,VMC,MANEUVERING,Preliminary,07/23/2018
1,20180717X23051,Accident,GAA18CA428,07/17/2018,"Nashville, TN",United States,36.124445,-86.678056,BNA,Nashville Intl,...,Instructional,,,,,,VMC,,Preliminary,07/19/2018
2,20180717X82154,Accident,CEN18LA276,07/17/2018,"Katy, TX",United States,29.868056,-95.795000,IWS,West Houston,...,Personal,,,,,1,VMC,,Preliminary,07/19/2018
3,20180716X44947,Accident,GAA18CA427,07/16/2018,"Hollywood, FL",United States,26.002222,-80.240000,HWO,NORTH PERRY,...,Instructional,,,,1,,VMC,,Preliminary,07/23/2018
4,20180716X60511,Accident,GAA18CA424,07/16/2018,"New Castle, IN",United States,39.564722,-85.235555,N/A,,...,Aerial Application,,,,1,,,,Preliminary,07/19/2018


In [7]:
df.describe()

,Event Id,Investigation Type,Accident Number,Event Date,Location,Country,Latitude,Longitude,Airport Code,Airport Name,...,Purpose of Flight,Air Carrier,Total Fatal Injuries,Total Serious Injuries,Total Minor Injuries,Total Uninjured,Weather Condition,Broad Phase of Flight,Report Status,Publication Date
count,81899,81899,81899,81899,81899,81899,81899,81899,81899,81899,...,81899,81899,81899,81899,81899,81899,81899,81899,81899,81899
unique,80704,2,81899,13203,25950,178,19101,20545,9860,23307,...,23,2958,124,41,64,371,4,13,4,3762
top,20100204X45658,Accident,FTW95LA369,07/08/2000,"ANCHORAGE, AK",United States,,,,,...,Personal,,0,0,0,1,VMC,LANDING,Probable Cause,
freq,3,78571,1,25,372,76901,53840,53849,35450,30345,...,45945,77836,40073,42647,40045,23328,72508,19925,75695,13906


In [8]:
df = df.replace('', np.nan)

In [9]:
df.describe()

,Event Id,Investigation Type,Accident Number,Event Date,Location,Country,Latitude,Longitude,Airport Code,Airport Name,...,Purpose of Flight,Air Carrier,Total Fatal Injuries,Total Serious Injuries,Total Minor Injuries,Total Uninjured,Weather Condition,Broad Phase of Flight,Report Status,Publication Date
count,81899,81899,81899,81899,81824,81392,28059,28050,46449,51554,...,77474,4063,56503,54055,55224,68447,79299,75487,81899,67993
unique,80704,2,81899,13203,25949,177,19100,20544,9859,23306,...,22,2957,123,40,63,370,3,12,4,3761
top,20100204X45658,Accident,FTW95LA369,07/08/2000,"ANCHORAGE, AK",United States,33.460833,-112.082500,NONE,N/A,...,Personal,UNITED AIRLINES,0,0,0,1,VMC,LANDING,Probable Cause,03/31/1993
freq,3,78571,1,25,372,76901,29,34,1473,2462,...,45945,49,40073,42647,40045,23328,72508,19925,75695,452


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81899 entries, 0 to 81898
Data columns (total 31 columns):
Event Id                  81899 non-null object
Investigation Type        81899 non-null object
Accident Number           81899 non-null object
Event Date                81899 non-null object
Location                  81824 non-null object
Country                   81392 non-null object
Latitude                  28059 non-null object
Longitude                 28050 non-null object
Airport Code              46449 non-null object
Airport Name              51554 non-null object
Injury Severity           81899 non-null object
Aircraft Damage           79321 non-null object
Aircraft Category         25157 non-null object
Registration Number       78414 non-null object
Make                      81828 non-null object
Model                     81800 non-null object
Amateur Built             81267 non-null object
Number of Engines         77419 non-null object
Engine Type               7

In [11]:
df.columns

Index(['Event Id', 'Investigation Type', 'Accident Number', 'Event Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport Code',
       'Airport Name', 'Injury Severity', 'Aircraft Damage',
       'Aircraft Category', 'Registration Number', 'Make', 'Model',
       'Amateur Built', 'Number of Engines', 'Engine Type', 'FAR Description',
       'Schedule', 'Purpose of Flight', 'Air Carrier', 'Total Fatal Injuries',
       'Total Serious Injuries', 'Total Minor Injuries', 'Total Uninjured',
       'Weather Condition', 'Broad Phase of Flight', 'Report Status',
       'Publication Date'],
      dtype='object')

In [12]:
df.columns = ['Event_Id', 'Investigation_Type', 'Accident_Number', 'Event_Date', 'Location',
              'Country', 'Latitude', 'Longitude', 'Airport_Code', 'Airport_Name', 'Injury_Severity',
              'Aircraft_Damage', 'Aircraft_Category', 'Registration_Number', 'Make', 'Model',
              'Amateur_Built', 'Number_Engines', 'Engine_Type', 'FAR_Description', 'Schedule',
              'Flight_Purpose', 'Air_Carrier', 'Total_Fatal_Injuries', 'Total_Serious_Injuries',
              'Total_Minor_Injuries', 'Total_Uninjured', 'Weather_Condition', 'Broad_Phase_of_Flight',
              'Report_Status', 'Publication_Date']

In [13]:
df

,Event_Id,Investigation_Type,Accident_Number,Event_Date,Location,Country,Latitude,Longitude,Airport_Code,Airport_Name,...,Flight_Purpose,Air_Carrier,Total_Fatal_Injuries,Total_Serious_Injuries,Total_Minor_Injuries,Total_Uninjured,Weather_Condition,Broad_Phase_of_Flight,Report_Status,Publication_Date
0,20180719X01751,Accident,CEN18LA281,07/19/2018,"Shreveport, LA",United States,32.540277,-93.745000,DTN,Shreveport Executive,...,Personal,NaN,NaN,NaN,NaN,1,VMC,MANEUVERING,Preliminary,07/23/2018
1,20180717X23051,Accident,GAA18CA428,07/17/2018,"Nashville, TN",United States,36.124445,-86.678056,BNA,Nashville Intl,...,Instructional,NaN,NaN,NaN,NaN,NaN,VMC,NaN,Preliminary,07/19/2018
2,20180717X82154,Accident,CEN18LA276,07/17/2018,"Katy, TX",United States,29.868056,-95.795000,IWS,West Houston,...,Personal,NaN,NaN,NaN,NaN,1,VMC,NaN,Preliminary,07/19/2018
3,20180716X44947,Accident,GAA18CA427,07/16/2018,"Hollywood, FL",United States,26.002222,-80.240000,HWO,NORTH PERRY,...,Instructional,NaN,NaN,NaN,1,NaN,VMC,NaN,Preliminary,07/23/2018
4,20180716X60511,Accident,GAA18CA424,07/16/2018,"New Castle, IN",United States,39.564722,-85.235555,N/A,NaN,...,Aerial Application,NaN,NaN,NaN,1,NaN,NaN,NaN,Preliminary,07/19/2018
5,20180715X83946,Accident,GAA18CA423,07/15/2018,"Lowville, NY",United States,43.766945,-75.600278,NaN,N/A,...,Personal,NaN,NaN,NaN,NaN,1,VMC,NaN,Preliminary,07/18/2018
6,20180716X21449,Accident,GAA18CA422,07/15/2018,"Palmer, AK",United States,61.595000,-149.088611,PAQ,Palmer Buddy Woods Muni,...,Personal,NaN,NaN,NaN,NaN,1,VMC,NaN,Preliminary,07/23/2018
7,20180715X13651,Accident,CEN18FA267,07/15/2018,"La Porte, TX",United States,29.674445,-95.056389,T41,La Porte Municipal Airport,...,Personal,NaN,2,NaN,NaN,NaN,VMC,APPROACH,Preliminary,07/19/2018
8,20180716X64029,Accident,CEN18LA271,07/14/2018,"Lexington, TX",United States,30.391666,-97.108333,TA75,Cotton Patch,...,Personal,NaN,NaN,NaN,NaN,1,VMC,TAXI,Preliminary,07/19/2018
9,20180712X20155,Accident,ERA18FA189,07/12/2018,"Plainville, CT",United States,41.683889,-72.870000,4B8,Robertson Airport,...,Personal,NaN,1,NaN,NaN,NaN,VMC,GO-AROUND,Preliminary,07/20/2018


In [14]:
df.Weather_Condition.unique()

array(['VMC', nan, 'IMC', 'UNK'], dtype=object)

## Weather condition definitions

VMC and IMC are aviation terms used to describe meteorological conditions during flight. VMC stands for visual meteorological conditions, and IMC stands for instrument meteorological conditions. 

The definition of IMC, according to the FAA, is meteorological conditions expressed in terms of visibility, distance from clouds, and ceiling less than the minima specified for visual meteorological conditions (VMC).

IMC is any meteorological condition that doesn’t qualify as visual meteorological conditions, or weather conditions worse than VMC.

UNK is most likely unknown.

## ACCIDENT VS INCIDENT

In aviation, an accident is defined by the Convention on International Civil Aviation Annex 13 as an occurrence associated with the operation of an aircraft, which takes place from the time any person boards the aircraft with the intention of flight until all such persons have disembarked, and in which a) a person is fatally or seriously injured, b) the aircraft sustains significant damage or structural failure, or c) the aircraft goes missing or becomes completely inaccessible.

Annex 13 defines an incident as an occurrence, other than an accident, associated with the operation of an aircraft which affects or could affect the safety of operation

In [15]:
df.Total_Uninjured.value_counts()

1      23328
0      19121
2      14765
3       4069
4       2547
5        842
6        458
7        262
8        145
9        119
10       103
11        58
12        48
13        40
16        38
15        36
14        32
53        32
17        32
19        28
18        27
20        25
21        22
131       20
140       20
50        19
44        19
52        19
51        18
133       18
       ...  
404        1
378        1
258        1
326        1
507        1
412        1
357        1
264        1
386        1
329        1
367        1
360        1
558        1
230        1
302        1
293        1
178        1
338        1
280        1
204        1
403        1
237        1
333        1
385        1
439        1
254        1
393        1
311        1
399        1
394        1
Name: Total_Uninjured, Length: 370, dtype: int64

In [16]:
df.Injury_Severity

0          Non-Fatal
1        Unavailable
2          Non-Fatal
3          Non-Fatal
4          Non-Fatal
5          Non-Fatal
6          Non-Fatal
7           Fatal(2)
8          Non-Fatal
9           Fatal(1)
10         Non-Fatal
11          Fatal(1)
12          Fatal(1)
13         Non-Fatal
14         Non-Fatal
15          Fatal(1)
16         Non-Fatal
17         Non-Fatal
18         Non-Fatal
19         Non-Fatal
20         Non-Fatal
21          Fatal(1)
22         Non-Fatal
23          Fatal(2)
24         Non-Fatal
25         Non-Fatal
26         Non-Fatal
27         Non-Fatal
28         Non-Fatal
29         Non-Fatal
            ...     
81869       Incident
81870       Fatal(2)
81871      Non-Fatal
81872      Non-Fatal
81873      Non-Fatal
81874       Fatal(8)
81875       Fatal(1)
81876      Non-Fatal
81877      Non-Fatal
81878      Non-Fatal
81879       Fatal(1)
81880       Fatal(1)
81881       Fatal(2)
81882      Non-Fatal
81883      Non-Fatal
81884      Non-Fatal
81885      No

In [17]:
state_df = pd.concat([df['Event_Id'], df['Location'].str.split(', ', expand=True)], axis=1)

In [18]:
state_df

,Event_Id,0,1,2,3,4
0,20180719X01751,Shreveport,LA,None,None,None
1,20180717X23051,Nashville,TN,None,None,None
2,20180717X82154,Katy,TX,None,None,None
3,20180716X44947,Hollywood,FL,None,None,None
4,20180716X60511,New Castle,IN,None,None,None
5,20180715X83946,Lowville,NY,None,None,None
6,20180716X21449,Palmer,AK,None,None,None
7,20180715X13651,La Porte,TX,None,None,None
8,20180716X64029,Lexington,TX,None,None,None
9,20180712X20155,Plainville,CT,None,None,None


In [19]:
state_df['City'] = state_df[0]
state_df['State'] = state_df[1]

state_df = state_df.drop([0,1,2,3,4],1)

In [20]:
state_df

,Event_Id,City,State
0,20180719X01751,Shreveport,LA
1,20180717X23051,Nashville,TN
2,20180717X82154,Katy,TX
3,20180716X44947,Hollywood,FL
4,20180716X60511,New Castle,IN
5,20180715X83946,Lowville,NY
6,20180716X21449,Palmer,AK
7,20180715X13651,La Porte,TX
8,20180716X64029,Lexington,TX
9,20180712X20155,Plainville,CT


In [21]:
df.reset_index(inplace=True)
state_df.reset_index(inplace=True)

In [22]:
state_df

,index,Event_Id,City,State
0,0,20180719X01751,Shreveport,LA
1,1,20180717X23051,Nashville,TN
2,2,20180717X82154,Katy,TX
3,3,20180716X44947,Hollywood,FL
4,4,20180716X60511,New Castle,IN
5,5,20180715X83946,Lowville,NY
6,6,20180716X21449,Palmer,AK
7,7,20180715X13651,La Porte,TX
8,8,20180716X64029,Lexington,TX
9,9,20180712X20155,Plainville,CT


In [23]:
df = pd.merge(df, state_df, on='index')

df = df.drop(['Location'],1)

In [24]:
df.set_index('index', inplace=True)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81899 entries, 0 to 81898
Data columns (total 33 columns):
Event_Id_x                81899 non-null object
Investigation_Type        81899 non-null object
Accident_Number           81899 non-null object
Event_Date                81899 non-null object
Country                   81392 non-null object
Latitude                  28059 non-null object
Longitude                 28050 non-null object
Airport_Code              46449 non-null object
Airport_Name              51554 non-null object
Injury_Severity           81899 non-null object
Aircraft_Damage           79321 non-null object
Aircraft_Category         25157 non-null object
Registration_Number       78414 non-null object
Make                      81828 non-null object
Model                     81800 non-null object
Amateur_Built             81267 non-null object
Number_Engines            77419 non-null object
Engine_Type               78016 non-null object
FAR_Description           2

In [26]:
df.Aircraft_Damage.value_counts()

Substantial    59046
Destroyed      17680
Minor           2595
Name: Aircraft_Damage, dtype: int64

In [27]:
df.Latitude = df.Latitude.astype('float')
df.Longitude = df.Longitude.astype('float')
df.Total_Fatal_Injuries = df.Total_Fatal_Injuries.astype('float')
df.Total_Serious_Injuries = df.Total_Serious_Injuries.astype('float')
df.Total_Minor_Injuries = df.Total_Minor_Injuries.astype('float')
df.Total_Uninjured = df.Total_Uninjured.astype('float')

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81899 entries, 0 to 81898
Data columns (total 33 columns):
Event_Id_x                81899 non-null object
Investigation_Type        81899 non-null object
Accident_Number           81899 non-null object
Event_Date                81899 non-null object
Country                   81392 non-null object
Latitude                  28059 non-null float64
Longitude                 28050 non-null float64
Airport_Code              46449 non-null object
Airport_Name              51554 non-null object
Injury_Severity           81899 non-null object
Aircraft_Damage           79321 non-null object
Aircraft_Category         25157 non-null object
Registration_Number       78414 non-null object
Make                      81828 non-null object
Model                     81800 non-null object
Amateur_Built             81267 non-null object
Number_Engines            77419 non-null object
Engine_Type               78016 non-null object
FAR_Description          

In [29]:
df.describe()

,Latitude,Longitude,Total_Fatal_Injuries,Total_Serious_Injuries,Total_Minor_Injuries,Total_Uninjured
count,28059.000000,28050.000000,56503.000000,54055.000000,55224.000000,68447.000000
mean,37.598733,-93.418193,0.827425,0.324262,0.510955,5.872485
std,12.277073,39.827566,6.216636,1.372970,2.775717,29.465923
min,-78.016945,-178.676111,0.000000,0.000000,0.000000,0.000000
25%,33.351667,-114.938194,0.000000,0.000000,0.000000,0.000000
50%,38.148889,-94.371389,0.000000,0.000000,0.000000,1.000000
75%,42.531111,-81.630069,1.000000,0.000000,1.000000,2.000000
max,89.218056,177.557778,349.000000,111.000000,380.000000,699.000000


In [30]:
df.Total_Uninjured.max

<bound method Series.max of index
0         1.0
1         NaN
2         1.0
3         NaN
4         NaN
5         1.0
6         1.0
7         NaN
8         1.0
9         NaN
10        1.0
11        NaN
12        NaN
13        3.0
14        1.0
15        NaN
16        1.0
17        1.0
18        1.0
19        2.0
20        2.0
21        NaN
22        3.0
23        NaN
24        1.0
25        1.0
26        NaN
27        1.0
28        NaN
29        1.0
         ... 
81869     1.0
81870     0.0
81871     0.0
81872     0.0
81873     1.0
81874     0.0
81875     0.0
81876     1.0
81877     2.0
81878     0.0
81879     0.0
81880     0.0
81881     0.0
81882     1.0
81883     1.0
81884     2.0
81885     1.0
81886     0.0
81887     2.0
81888     2.0
81889     0.0
81890     1.0
81891     1.0
81892     0.0
81893    44.0
81894     NaN
81895     0.0
81896     NaN
81897     0.0
81898     0.0
Name: Total_Uninjured, Length: 81899, dtype: float64>

In [31]:
df.describe()

,Latitude,Longitude,Total_Fatal_Injuries,Total_Serious_Injuries,Total_Minor_Injuries,Total_Uninjured
count,28059.000000,28050.000000,56503.000000,54055.000000,55224.000000,68447.000000
mean,37.598733,-93.418193,0.827425,0.324262,0.510955,5.872485
std,12.277073,39.827566,6.216636,1.372970,2.775717,29.465923
min,-78.016945,-178.676111,0.000000,0.000000,0.000000,0.000000
25%,33.351667,-114.938194,0.000000,0.000000,0.000000,0.000000
50%,38.148889,-94.371389,0.000000,0.000000,0.000000,1.000000
75%,42.531111,-81.630069,1.000000,0.000000,1.000000,2.000000
max,89.218056,177.557778,349.000000,111.000000,380.000000,699.000000


In [57]:
df.head()

,Event_Id_x,Investigation_Type,Accident_Number,Event_Date,Country,Latitude,Longitude,Airport_Code,Airport_Name,Injury_Severity,...,Total_Serious_Injuries,Total_Minor_Injuries,Total_Uninjured,Weather_Condition,Broad_Phase_of_Flight,Report_Status,Publication_Date,Event_Id_y,City,State
index,,,,,,,,,,,,,,,,,,,,,
0,20180719X01751,Accident,CEN18LA281,07/19/2018,United States,32.540277,-93.745000,DTN,Shreveport Executive,Non-Fatal,...,NaN,NaN,1.0,VMC,MANEUVERING,Preliminary,07/23/2018,20180719X01751,Shreveport,LA
1,20180717X23051,Accident,GAA18CA428,07/17/2018,United States,36.124445,-86.678056,BNA,Nashville Intl,Unavailable,...,NaN,NaN,NaN,VMC,NaN,Preliminary,07/19/2018,20180717X23051,Nashville,TN
2,20180717X82154,Accident,CEN18LA276,07/17/2018,United States,29.868056,-95.795000,IWS,West Houston,Non-Fatal,...,NaN,NaN,1.0,VMC,NaN,Preliminary,07/19/2018,20180717X82154,Katy,TX
3,20180716X44947,Accident,GAA18CA427,07/16/2018,United States,26.002222,-80.240000,HWO,NORTH PERRY,Non-Fatal,...,NaN,1.0,NaN,VMC,NaN,Preliminary,07/23/2018,20180716X44947,Hollywood,FL
4,20180716X60511,Accident,GAA18CA424,07/16/2018,United States,39.564722,-85.235555,N/A,NaN,Non-Fatal,...,NaN,1.0,NaN,NaN,NaN,Preliminary,07/19/2018,20180716X60511,New Castle,IN


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81899 entries, 0 to 81898
Data columns (total 33 columns):
Event_Id_x                81899 non-null object
Investigation_Type        81899 non-null object
Accident_Number           81899 non-null object
Event_Date                81899 non-null object
Country                   81392 non-null object
Latitude                  28059 non-null float64
Longitude                 28050 non-null float64
Airport_Code              46449 non-null object
Airport_Name              51554 non-null object
Injury_Severity           81899 non-null object
Aircraft_Damage           79321 non-null object
Aircraft_Category         25157 non-null object
Registration_Number       78414 non-null object
Make                      81828 non-null object
Model                     81800 non-null object
Amateur_Built             81267 non-null object
Number_Engines            77419 non-null object
Engine_Type               78016 non-null object
FAR_Description          

In [58]:
df['Broad_Phase_of_Flight'].value_counts()

LANDING        19925
TAKEOFF        15767
CRUISE         10911
MANEUVERING    10115
APPROACH        7940
TAXI            2401
CLIMB           2333
DESCENT         2264
GO-AROUND       1667
STANDING        1262
UNKNOWN          738
OTHER            164
Name: Broad_Phase_of_Flight, dtype: int64

In [40]:
df['Aircraft_Category'].value_counts()

Airplane             21555
Helicopter            2658
Glider                 420
Balloon                188
Gyrocraft               89
Weight-Shift            85
Powered Parachute       58
Gyroplane               44
Ultralight              30
Unknown                 19
Powered-Lift             6
Blimp                    4
Rocket                   1
Name: Aircraft_Category, dtype: int64

In [41]:
df['Aircraft_Damage'].value_counts()

Substantial    59046
Destroyed      17680
Minor           2595
Name: Aircraft_Damage, dtype: int64

In [47]:
print(len(df['Make'].unique()))

7845


In [ ]:
df['Make'].unique()

In [50]:
df['Make'] = df['Make'].str.upper()

In [51]:
print(len(df['Make'].unique()))

7050


In [54]:
df['Amateur_Built'].value_counts()

No     73302
Yes     7965
Name: Amateur_Built, dtype: int64

In [55]:
df['Schedule'].value_counts()

UNK     4099
NSCH    3962
SCHD    3634
Name: Schedule, dtype: int64

In [56]:
df['Flight_Purpose'].value_counts()

Personal                     45945
Instructional                 9799
Unknown                       6785
Aerial Application            4468
Business                      3926
Positioning                   1542
Other Work Use                1140
Ferry                          790
Public Aircraft                712
Aerial Observation             703
Executive/Corporate            522
Flight Test                    339
Skydiving                      166
Air Race/Show                  155
Public Aircraft - Federal       93
External Load                   92
Banner Tow                      86
Public Aircraft - State         64
Public Aircraft - Local         62
Glider Tow                      49
Firefighting                    24
Air Drop                        12
Name: Flight_Purpose, dtype: int64

In [32]:
no_deaths = df[df['Total_Fatal_Injuries'] == 0]

no_deaths

,Event_Id_x,Investigation_Type,Accident_Number,Event_Date,Country,Latitude,Longitude,Airport_Code,Airport_Name,Injury_Severity,...,Total_Serious_Injuries,Total_Minor_Injuries,Total_Uninjured,Weather_Condition,Broad_Phase_of_Flight,Report_Status,Publication_Date,Event_Id_y,City,State
index,,,,,,,,,,,,,,,,,,,,,
15834,20090306X41211,Accident,ERA09WA183,03/04/2009,Netherlands Antilles,18.035555,-63.101111,NaN,NaN,Non-Fatal,...,0.0,0.0,3.0,VMC,NaN,Foreign,03/18/2009,20090306X41211,St. Maarten,Netherlands Antilles
15901,20090304X25224,Accident,WPR09WA138,02/18/2009,United Kingdom,NaN,NaN,NaN,NaN,Non-Fatal,...,0.0,0.0,18.0,VMC,NaN,Foreign,03/04/2009,20090304X25224,North Sea,United Kingdom
15914,20090225X14913,Accident,ERA09WA174,02/14/2009,Peru,-14.035555,-75.755278,SPIA,Las Dunas,Non-Fatal,...,0.0,3.0,0.0,VMC,NaN,Foreign,03/02/2009,20090225X14913,Ica,Peru
16025,20090325X84835,Incident,ERA09WA212,01/10/2009,Brazil,-17.834444,-50.955556,NaN,NaN,Incident,...,0.0,0.0,2.0,NaN,NaN,Foreign,03/27/2009,20090325X84835,Rio Verde,Brazil
16068,20081230X72234,Accident,ERA09LA114,12/29/2008,United States,37.660556,-87.256944,NaN,NaN,Non-Fatal,...,0.0,0.0,1.0,VMC,NaN,Probable Cause,03/23/2009,20081230X72234,Owensboro,KY
16086,20081229X40417,Accident,CEN09LA106,12/26/2008,United States,NaN,NaN,KMCI,Kansas City International,Non-Fatal,...,1.0,0.0,76.0,VMC,DESCENT,Probable Cause,02/25/2009,20081229X40417,Kansas City,MO
16131,20081215X93019,Incident,DCA09IA017,12/14/2008,United States,39.871945,-75.241111,PHL,Philadelphia IAP,Incident,...,0.0,0.0,3.0,NaN,NaN,Preliminary,12/19/2008,20081215X93019,Philadelphia,PA
16185,20081126X30503,Accident,WPR09LA046,11/25/2008,United States,36.586944,-121.842777,MRY,Monterey Peninsula Airport,Non-Fatal,...,1.0,1.0,0.0,VMC,DESCENT,Probable Cause,03/23/2009,20081126X30503,Monterey,CA
16194,20081124X54353,Accident,CEN09WA073,11/22/2008,Canada,54.558889,-94.491389,CYGO,God's Lake Narrows Airport,Non-Fatal,...,0.0,5.0,0.0,VMC,NaN,Foreign,11/26/2008,20081124X54353,God's Lake Narrows,Canada


In [35]:
fatal_flights = df[df['Total_Fatal_Injuries'] > 0]

fatal_flights

,Event_Id_x,Investigation_Type,Accident_Number,Event_Date,Country,Latitude,Longitude,Airport_Code,Airport_Name,Injury_Severity,...,Total_Serious_Injuries,Total_Minor_Injuries,Total_Uninjured,Weather_Condition,Broad_Phase_of_Flight,Report_Status,Publication_Date,Event_Id_y,City,State
index,,,,,,,,,,,,,,,,,,,,,
7,20180715X13651,Accident,CEN18FA267,07/15/2018,United States,29.674445,-95.056389,T41,La Porte Municipal Airport,Fatal(2),...,NaN,NaN,NaN,VMC,APPROACH,Preliminary,07/19/2018,20180715X13651,La Porte,TX
9,20180712X20155,Accident,ERA18FA189,07/12/2018,United States,41.683889,-72.870000,4B8,Robertson Airport,Fatal(1),...,NaN,NaN,NaN,VMC,GO-AROUND,Preliminary,07/20/2018,20180712X20155,Plainville,CT
11,20180712X55625,Accident,CEN18FA265,07/12/2018,United States,NaN,NaN,7TS0,Fairview Airport,Fatal(1),...,NaN,NaN,NaN,VMC,UNKNOWN,Preliminary,07/19/2018,20180712X55625,Rhome,TX
12,20180711X93304,Accident,CEN18FA263,07/11/2018,United States,34.946944,-91.114167,NaN,N/A,Fatal(1),...,NaN,NaN,NaN,VMC,MANEUVERING,Preliminary,07/19/2018,20180711X93304,Wheatley,AR
15,20180710X25659,Accident,CEN18FA262,07/10/2018,United States,34.978056,-106.000000,0E0,Moriarty Municipal Airport,Fatal(1),...,NaN,NaN,NaN,NaN,MANEUVERING,Preliminary,07/19/2018,20180710X25659,Moriarty,NM
21,20180710X03854,Accident,ERA18LA188,07/09/2018,United States,34.522223,-85.986389,NaN,N/A,Fatal(1),...,NaN,NaN,NaN,VMC,LANDING,Preliminary,07/20/2018,20180710X03854,Section,AL
23,20180708X12242,Accident,ERA18FA187,07/08/2018,United States,37.288333,-76.729445,JGG,Williamsburg-Jamestown,Fatal(2),...,NaN,NaN,NaN,VMC,CRUISE,Preliminary,07/20/2018,20180708X12242,Williamsburg,VA
31,20180706X04105,Accident,CEN18FA258,07/06/2018,United States,39.633056,-85.635833,NaN,N/A,Fatal(1),...,NaN,NaN,NaN,VMC,MANEUVERING,Preliminary,07/11/2018,20180706X04105,Arlington,IN
37,20180705X51303,Accident,WPR18LA188,07/04/2018,United States,-1.560000,167.560000,NaN,N/A,Fatal(2),...,NaN,NaN,NaN,NaN,TAKEOFF,Preliminary,07/23/2018,20180705X51303,Majuro,MH


In [38]:
no_deaths['Investigation_Type'].value_counts()

Accident    38497
Incident     1576
Name: Investigation_Type, dtype: int64

In [37]:
fatal_flights['Investigation_Type'].value_counts()

Accident    16411
Incident       19
Name: Investigation_Type, dtype: int64

In [258]:
df['Total_passengers'] = df[df['Total_Fatal_Injuries'] + df['Total_Serious_Injuries'] == 0 & df['Total_Minor_Injuries'] == 0 & df['Total_Uninjured'] == 0]

TypeError: cannot compare a dtyped [float64] array with a scalar of type [bool]

In [257]:
no_injuries.describe()

,Latitude,Longitude,Total_Fatal_Injuries,Total_Serious_Injuries,Total_Minor_Injuries,Total_Uninjured
count,245.000000,245.000000,19116.000000,19115.000000,19117.000000,19121.0
mean,35.914178,-96.214672,1.145951,0.447397,0.585657,0.0
std,12.765833,30.141648,5.373844,1.116427,1.223099,0.0
min,-33.223611,-157.511667,0.000000,0.000000,0.000000,0.0
25%,32.557222,-117.966667,0.000000,0.000000,0.000000,0.0
50%,36.586944,-97.003333,0.000000,0.000000,0.000000,0.0
75%,40.869445,-81.337222,1.000000,1.000000,1.000000,0.0
max,87.145833,28.715278,270.000000,81.000000,63.000000,0.0
